## 1. Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=e01cbd30329fa9122d819cadac8e5199ac5107df09419c6992578e8ab55d2063
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [29]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import os
import torch
from tqdm import tqdm
# from datasets import metric
import evaluate

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
MAX_LENGTH=500
BATCH_SIZE=16

## 2. Load Dataset & Prepocess the Data
- Load Train and Test dataset
- Truncate the answers to MAX_LENGTH.
- Concat Question and Answer using special token, and add start and end token.
-  Repeate the above two steps for Train and Test


In [32]:
train_test_dict={"train":{},"test":{}}

In [33]:
tokenizer=AutoTokenizer.from_pretrained('distilgpt2', add_special_tokens=True,additional_special_tokens=['[response]'], pad_token="[Pad]",padding_side='left')

In [34]:
bos_token=tokenizer.decode(tokenizer.bos_token_id)
eos_token=tokenizer.decode(tokenizer.eos_token_id)

In [35]:
for key in train_test_dict:
    dataset_pd=pd.read_csv(f'/content/drive/MyDrive/NLP_Project/{key}_datasets/MedQuAD_{key}.csv')
    dataset_pd=dataset_pd.astype('string')
    dataset_pd['Question']=bos_token + dataset_pd['Question'] + ' [response] '
    dataset_pd['QA_pairs'] = dataset_pd['Question']  + dataset_pd['Answer_cut'] + eos_token
    dataset_list=list(dataset_pd['QA_pairs'])
    questions_list=list(dataset_pd['Question'])
    train_test_dict[key]['dataset']=dataset_pd
    train_test_dict[key]['question_list']=questions_list
    train_test_dict[key]['QA_list']=dataset_list

## 3. Initialize pytorch dataset and dataloaders

In [36]:
class MedicalDataset(Dataset):
    def __init__(self, data,  tokenizer, questions):
        self.tokenizer = tokenizer
        self.data=self.tokenizer(data, padding=True, truncation=True ,return_tensors='pt')
        self.non_tokenized_data= data
        self.tokenized_questions=self.tokenizer(questions, padding=True, truncation=False, return_tensors='pt')
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        return self.data['input_ids'][idx], self.data['attention_mask'][idx], self.non_tokenized_data[idx],self.tokenized_questions['input_ids'][idx], self.tokenized_questions['attention_mask'][idx]



In [37]:
train_dataset = MedicalDataset(train_test_dict['train']['QA_list'], tokenizer, train_test_dict['train']['question_list'])
test_dataset = MedicalDataset(train_test_dict['test']['QA_list'], tokenizer, train_test_dict['test']['question_list'])


In [18]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

## 4. Define the model
- Resize the total number of model's embeddings  as we added a special token ('['response']') for sperating the question from the answer.
- Set the model's hyperparams, optimizer and the learning rate scheduler.

In [19]:
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
model.resize_token_embeddings(len(tokenizer))
model=model.to(device)

In [20]:
epochs = 3
learning_rate = 5e-4
warmup_steps = 1e2
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler=get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warmup_steps, num_training_steps= epochs*(len(train_loader)))
grad_accumulatio_steps=2

## 6. Model Training

In [ ]:
from datetime import datetime

date_string = 'fine_tune_model_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
train_loss_values=[]
test_loss_values=[]
ctr=1
for epoch in range(epochs): # number of epochs
    model.train()
    train_loss=0
    model.train()
    for input_ids, attention_mask,raw_data, toknized_questions_id,toknized_questions_mask in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids=input_ids.to(device)
        attention_mask=attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        train_loss+=loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        # if (grad_accumulatio_steps != 0) and  ((grad_accumulatio_steps %grad_accumulatio_steps) ==0):
        #     optimizer.step()
        #     scheduler.step()
        #     ctr+=1
    model.eval()
    val_loss=0
    with torch.no_grad():
      for input_ids, attention_mask, raw_data, toknized_questions_id,toknized_questions_mask in tqdm(test_loader):
        input_ids=input_ids.to(device)
        attention_mask=attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        val_loss += outputs.loss.item()
    avg_val_loss = val_loss / len(test_loader)
    test_loss_values.append(avg_val_loss)


    avg_train_loss = train_loss / len(train_loader)
    train_loss_values.append(avg_train_loss)

    print(f"Epoch {epoch}, train_loss: {avg_train_loss}, val_loss: {avg_val_loss}")


In [38]:
model.save_pretrained('/content/drive/MyDrive/NLP_Project/models/fineTune_models/MedQuad_left_pad_Latest_fineTuned_gpt2_3epoch'+date_string)

## 7. Model Evaluation
- Generating the sacreBleu and Rouge scores on test datasset.



In [39]:
model=GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/NLP_Project/models/fineTune_models/MedQuad_left_pad_Latest_fineTuned_gpt2_3epochfine_tune_model_2024_04_06_14_18_29/').to(device)


In [45]:
model.eval()
bleu=evaluate.load('sacrebleu')
rouge=evaluate.load('rouge')

for _, _, raw_data, toknized_questions_id,toknized_questions_mask in tqdm(test_loader):
    input_ids=toknized_questions_id.to(device)
    attention_mask=toknized_questions_mask.to(device)
    answers=[qa.split('[response]')[-1] for qa in raw_data]
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask,  pad_token_id=tokenizer.pad_token_id,max_new_tokens=MAX_LENGTH)
    decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    for pred, ref in zip(answers, decoded_outputs):
        bleu.add(references = ref, predictions=pred)
        rouge.add(references = ref, predictions=pred)

bleu=bleu.compute()
rouge=rouge.compute()




print(f"Bleu: {bleu['score']},  Rouge:{rouge['rouge1']}, Rouge:{rouge['rougeL']}")

100%|██████████| 205/205 [10:17<00:00,  3.01s/it]


Bleu: 6.27,  Rouge:0.208, Rouge:0.175


### 7.3) Chekcing the reponse for any one question.

In [47]:
output_1391=model.generate(input_ids=torch.tensor(tokenizer(questions_list[11])['input_ids']).reshape(1,-1).to(device),attention_mask=torch.tensor(tokenizer(questions_list[11])['attention_mask']).reshape(1,-1).to(device), pad_token_id=tokenizer.pad_token_id,max_new_tokens=MAX_LENGTH)

In [48]:
tokenizer.decode(tokenizer(bos_token+questions_list[11])['input_ids'][1:],skip_special_tokens=False)

'<|endoftext|>What to do for Gastritis? [response] '

In [49]:
tokenizer.decode(output_1391[0], skip_special_tokens=False)

'<|endoftext|>What to do for Gastritis? [response]  Gastritis is a disease in which the body does not make enough of a protein called a substance called a protein.  The body does not make enough of a protein called a substance called a protein. The body does not make enough of a protein called a substance called a protein. Gastritis is a disease in which the body does not make enough of a protein called a substance called a protein. The body does not make enough of a protein called a substance called a protein. Gastritis is a disease in which the body does not make enough of a protein called a substance. The body does not make enough of a protein called a substance. Gastritis is a disease in which the body does not make enough of a protein called a substance. Gastritis is a disease in which the body does not make enough of a protein called a substance. Gastritis is a disease in which the body does not make enough of a protein called a substance. Gastritis is a disease in which the body